In [1]:
from deap import gp
import numpy as np
import os
import sys
sys.path.append(os.path.abspath('../..'))  # or the full path to the "project" directory. This hack should be really fixed
from gpbr.gp.funcs import pow2, sqrtabs, expplusone


pset = gp.PrimitiveSet("main", 1)
pset.addPrimitive(np.add, 2)
# pset.addPrimitive(np.subtract, 2)
pset.addPrimitive(np.multiply, 2)
pset.addPrimitive(np.cos, 1)
pset.addPrimitive(np.sin, 1)
pset.addPrimitive(sqrtabs, 1)
# pset.addPrimitive(pow2, 1)
pset.addPrimitive(expplusone, 1)
pset.addEphemeralConstant('rand', (np.random.rand, 1)[0])
# pset.addTerminal(np.pi, 'pi')

pset.renameArguments(ARG0="s")

In [2]:
from deap import creator, base, tools, algorithms
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMin)

In [3]:
toolbox = base.Toolbox()
toolbox.register('expr', gp.genHalfAndHalf, pset=pset, min_=3, max_=6)
toolbox.register('individual', tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register('population', tools.initRepeat, list, toolbox.individual)
toolbox.register('compile', gp.compile, pset=pset)

In [21]:
ind = toolbox.individual()
print(ind)

expplusone(multiply(cos(expplusone(cos(sin(s)))), multiply(sqrtabs(multiply(sqrtabs(0.8444483164503179), multiply(s, s))), cos(expplusone(cos(0.3347829874291335))))))


In [5]:
test_set = np.linspace(0, 2*np.pi, 10)
test_set

array([0.        , 0.6981317 , 1.3962634 , 2.0943951 , 2.7925268 ,
       3.4906585 , 4.1887902 , 4.88692191, 5.58505361, 6.28318531])

In [6]:
ind_compiled =toolbox.compile(expr=ind)
print(ind_compiled(0.5))  # example of use

6.933346786355914


In [7]:
ind_compiled(test_set)

6.933346786355914

In [96]:
from deap.gp import PrimitiveTree, PrimitiveSet, Primitive, Terminal
import numpy as np

def evaluate_subtrees(tree: PrimitiveTree, pset: PrimitiveSet, dim:int,  **kwargs):
    """
    Evaluate the GP tree and compute results for every subtree using a stack-based approach,
    inspired by the __str__ method in PrimitiveTree.
    
    :param tree: deap.gp.PrimitiveTree, the tree to evaluate.
    :param pset: deap.gp.PrimitiveSet, the primitive set used to define functions and terminals.
    :param args: Variable arguments representing the inputs for ARG0, ARG1, etc.
    :return: A tuple (root_value, subtree_values), where root_value is the result of the entire tree,
             and subtree_values is a list where subtree_values[i] is the result of the subtree rooted at tree[i].
    """
    subtree_values = np.empty((len(tree), dim), dtype=np.float64)

    stack = []
    
    for i in range(len(tree)):
        stack.append((i, []))
        
        while stack and len(stack[-1][1]) == tree[stack[-1][0]].arity:
            idx, child_vals = stack.pop()
            node: Primitive | Terminal = tree[idx]
            
            if node.arity == 0:  # Terminal
                if node.name.startswith('ARG'):
                    val = np.array(kwargs[node.value], dtype=np.float64)
                else:
                    val = np.full(dim, node.value, dtype=np.float64)
            else:  # Primitive
                func = pset.context[node.name]
                val = np.asanyarray(func(*child_vals), dtype=np.float64)
            
            subtree_values[idx] = val
            
            if stack:
                stack[-1][1].append(val)

    return subtree_values

In [97]:
subtree_vals =evaluate_subtrees(ind, pset,dim=len(test_set), s=test_set)

In [98]:
subtree_vals[0]

array([  2.71828183,   4.4611056 ,   2.75545981,   5.52722692,
        13.06466705,  19.3441349 ,  11.23880426,   2.85064463,
       143.0469685 ,  21.19043583])

In [99]:
subtree_vals

array([[ 2.71828183e+00,  4.46110560e+00,  2.75545981e+00,
         5.52722692e+00,  1.30646670e+01,  1.93441349e+01,
         1.12388043e+01,  2.85064463e+00,  1.43046969e+02,
         2.11904358e+01],
       [ 0.00000000e+00,  4.95396628e-01,  1.35843297e-02,
         7.09686228e-01,  1.56991141e+00,  1.96238927e+00,
         1.41937246e+00,  4.75451539e-02,  3.96317303e+00,
         2.05354994e+00],
       [ 4.48356242e-01,  9.73449681e-01,  1.33465396e-02,
         4.64842238e-01,  7.71215264e-01,  7.71215264e-01,
         4.64842238e-01,  1.33465396e-02,  9.73449681e-01,
         4.48356242e-01],
       [ 7.38905610e+00,  6.05223664e+00,  4.72573592e+00,
         5.19584540e+00,  6.97323362e+00,  6.97323362e+00,
         5.19584540e+00,  4.72573592e+00,  6.05223664e+00,
         7.38905610e+00],
       [ 1.00000000e+00,  8.00427896e-01,  5.53023298e-01,
         6.47859345e-01,  9.42079051e-01,  9.42079051e-01,
         6.47859345e-01,  5.53023298e-01,  8.00427896e-01,
         1.

In [100]:
np.isfinite(subtree_vals).all()

True

In [31]:
np.isfinite(subtree_vals.flatten()).all()

TypeError: ufunc 'isfinite' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [50]:
for el in ind:
    print(el.name)

sin
sqrtabs
expplusone
add
ARG0
ARG0


In [66]:
def is_2pi_periodic(values, tolerance=1e-5):
    if isinstance(values, (np.integer, int, float)):
        return True
    return np.allclose(values[0], values[-1], atol=tolerance)

In [70]:
for el in subtree_vals:
    print(is_2pi_periodic(el))

True
True
True
True
True
False
True
True


In [85]:
str(ind)

'cos(cos(expplusone(add(sin(s), sin(0.5960927048090293)))))'

In [87]:
[str(gp.PrimitiveTree(ind[ind.searchSubtree(i)])) for i in range(len(ind))]

['cos(cos(expplusone(add(sin(s), sin(0.5960927048090293)))))',
 'cos(expplusone(add(sin(s), sin(0.5960927048090293))))',
 'expplusone(add(sin(s), sin(0.5960927048090293)))',
 'add(sin(s), sin(0.5960927048090293))',
 'sin(s)',
 's',
 'sin(0.5960927048090293)',
 '0.5960927048090293']

In [79]:
ind.searchSubtree(2)
ind[ind.searchSubtree(4)]

[<deap.gp.Primitive at 0x1bd57264180>, <deap.gp.Terminal at 0x1bd57253240>]

In [ ]:
# Test: compare DEAP compiled subtree outputs with custom evaluate_subtrees
import numpy as np
import deap.gp as gp
import traceback

# import project helpers
from gpbr.gp.funcs import sqrtabs, expplusone
from gpbr.gp.evaluators import evaluate_subtrees

# Build primitive set similar to the notebook
pset = gp.PrimitiveSet("main", 1)
pset.addPrimitive(np.add, 2)
pset.addPrimitive(np.multiply, 2)
pset.addPrimitive(np.cos, 1)
pset.addPrimitive(np.sin, 1)
pset.addPrimitive(sqrtabs, 1)
pset.addPrimitive(expplusone, 1)
pset.addEphemeralConstant('rand', lambda: np.random.rand())
pset.renameArguments(ARG0='s')

# Test inputs
test_set = np.linspace(0, 2*np.pi, 10)

# Example expressions (use argument name 's' because pset was renamed)
examples = [
    "add(sin(s), 1.0)",
    "multiply(s, cos(s))",
    "expplusone(s)",
    "add(multiply(s, s), 0.5)",
    "sqrtabs(s)",
    "add(add(sin(s), cos(s)), multiply(s, 0.1))",
]

print(f"Running {len(examples)} examples; test_set shape: {test_set.shape}")

for expr in examples:
    print('\nExpression:', expr)
    try:
        tree = gp.PrimitiveTree.from_string(expr, pset)
    except Exception as e:
        print('  Failed to parse into PrimitiveTree:', e)
        traceback.print_exc()
        continue

    # Evaluate all subtrees using the custom evaluator
    try:
        root_val, subtree_vals = evaluate_subtrees(tree, pset, s=test_set)
    except Exception as e:
        print('  evaluate_subtrees failed:', e)
        traceback.print_exc()
        continue

    all_ok = True
    for i in range(len(tree)):
        node = tree[i]
        sl = tree.searchSubtree(i)
        subtree = gp.PrimitiveTree(tree[sl])

        # Compile subtree with DEAP
        try:
            compiled_fn = gp.compile(subtree, pset)
        except Exception as e:
            print(f"  Failed to compile subtree {i} ({node.name}):", e)
            traceback.print_exc()
            all_ok = False
            continue

        # Evaluate compiled subtree
        try:
            compiled_val = compiled_fn(test_set)
        except Exception as e:
            # Some compiled functions might expect scalar input — try elementwise
            try:
                compiled_val = np.array([compiled_fn(x) for x in test_set])
            except Exception as e2:
                print(f"  Failed to run compiled subtree {i} ({node.name}):", e)
                traceback.print_exc()
                all_ok = False
                continue

        eval_val = subtree_vals[i]

        # Normalize to numpy arrays
        a = np.asarray(compiled_val)
        b = np.asarray(eval_val)

        # Compare shapes and numeric closeness
        shapes_match = a.shape == b.shape
        nums_match = False
        if shapes_match:
            try:
                nums_match = np.allclose(a, b, equal_nan=True)
            except Exception:
                nums_match = False

        if not (shapes_match and nums_match):
            all_ok = False
            print(f"  MISMATCH at node {i} ({node.name}): compiled shape {a.shape}, evaluator shape {b.shape}")
            # show short samples
            def sample(arr):
                try:
                    arr = np.asarray(arr)
                    if arr.size > 5:
                        return arr.flatten()[:5]
                    return arr
                except Exception:
                    return str(arr)
            print('    compiled sample =', sample(a))
            print('    evaluator sample=', sample(b))

    print('  OK' if all_ok else '  FAIL')

print('\nDone.')